In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('insurance.csv')

In [8]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [9]:

df.region.replace({'southwest':0, 'southeast':1, 'northwest':2, 'northeast':3})
df.sex.replace({'female':0, 'male':1})
df.smoker.replace({'yes':1, 'no':0})

0       1
1       0
2       0
3       0
4       0
       ..
1333    0
1334    0
1335    0
1336    0
1337    1
Name: smoker, Length: 1338, dtype: int64

In [38]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
import pandas as pd

def preprocess(path_to_data):

    df = pd.read_csv(path_to_data)

    X, y = df.values[:, :-1], df.values[:, -1]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    c_transformer = ColumnTransformer(
    [("ordinal",OrdinalEncoder(
    handle_unknown= 'use_encoded_value', unknown_value=-1)
    ,[1,4,5]), 
    ('non_transformed','passthrough',[0,2,3])])

    X_train = c_transformer.fit_transform(X_train)
    X_test = c_transformer.transform(X_test)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test, c_transformer, scaler 


preprocess('insurance.csv')

(array([[-0.97676557, -0.50466988, -1.40820029, -1.5330973 , -0.40713453,
         -0.89833872],
        [-0.97676557, -0.50466988,  1.31322188, -0.03364163,  0.32855417,
         -0.89833872],
        [-0.97676557, -0.50466988,  0.40608116,  0.89459283,  2.56690911,
          3.25603402],
        ...,
        [ 1.02378711, -0.50466988,  0.40608116,  0.03776102, -0.91016269,
         -0.89833872],
        [ 1.02378711, -0.50466988, -0.50105957, -1.46169465,  0.76659782,
         -0.89833872],
        [-0.97676557, -0.50466988,  1.31322188, -0.46205754, -1.96596021,
         -0.06746417]]),
 array([[ 1.02378711, -0.50466988,  1.31322188,  0.89459283, -0.08863026,
         -0.06746417],
        [-0.97676557, -0.50466988,  0.40608116,  0.53757957, -0.22180837,
         -0.06746417],
        [ 1.02378711,  1.98149332, -0.50105957,  0.60898222,  1.57449152,
          0.76341038],
        ...,
        [ 1.02378711, -0.50466988,  0.40608116,  1.10880078,  1.20785059,
         -0.89833872],
  

In [52]:
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor


def train_model():
    
    x_train, x_test, y_train, y_test, c_transformer, scaler = preprocess("insurance.csv")
    
    xgb_r = XGBRegressor()
    gboosting_r = GradientBoostingRegressor(learning_rate=0.01, n_estimators=50)
    dtree_r = DecisionTreeRegressor(random_state=42, max_depth=4 )
    
    xgb_r.fit(x_train,y_train)
    gboosting_r.fit(x_train, y_train)
    dtree_r.fit(x_train, y_train)
    
    return xgb_r , gboosting_r, dtree_r, c_transformer, scaler

train_model()


(XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None),
 GradientBoostingRegressor(learning_rate=0.01, n_estimators=50),
 DecisionTreeRegressor(max_depth=4, random_state=42),
 ColumnTransformer(transformers=[('ordinal',
                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                 unknown_value=-1),
                                  [1, 4, 5])

In [55]:
import pandas as pd
import numpy as np
model = train_model()
from sklearn.compose import ColumnTransformer

while True:

    age = 25#int(input("How old are you? \n"))
    sex = 'm' #str(input("Which is your sex? If you are a male answer 'm', if you are a female answer with an 'f': \n"))
    if sex.lower() == 'm':
        sex = 'male'
    else:
        sex = 'female'
    bmi = 27 #float(input("Which is your bmi ('Body mass index')? \n"))
    child = 3# int(input("How many children do you have? \n"))
    smoke = 'no' #str(input("Do you smoke? Options: yes,no \n"))
    region = 'northeast' #str(input("Which is your residential area in the US? Options: northeast, southeast, southwest, northwest \n"))

    #Preprocess
    ct = model[-2]
    scaler = model[-1]
    X = pd.DataFrame({"age":age, "sex":sex, "bmi":bmi, "child":child, "smoke":smoke," region":region}, index=[0])
    X_trans = ct.transform(X)
    X_scaled = scaler.transform(X_trans)

    #Predict

    predictions = np.array([clf.predict(X_scaled) for clf in model[:-2]])
    a = predictions.mean()
    # print(a)
    # print(f"Prediction for your conditions: {predictions.mean()}")

5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.650052475753
5787.65005

KeyboardInterrupt: 